In [ ]:
!pip install nltk
!pip install keras
!pip install tensorflow

In [4]:
dataset0 = pd.read_csv("input/clickbait_data.csv")

In [5]:
dataset0 = dataset0.rename(columns={"headline": "text", "clickbait": "label"})

print(dataset0.head())
print(dataset0.shape)

                                                text  label
0                                 Should I Get Bings      1
1      Which TV Female Friend Group Do You Belong In      1
2  The New "Star Wars: The Force Awakens" Trailer...      1
3  This Vine Of New York On "Celebrity Big Brothe...      1
4  A Couple Did A Stunning Photo Shoot With Their...      1
(32000, 2)


In [6]:
# pre process text ( remove punctuation, make characters lowercase and do lemmatization )


def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # 1. Lowercase the text
    text = text.lower()

    # 2. Remove punctuation using regex
    text = re.sub(rf"[{re.escape(string.punctuation)}]", "", text)

    # 3. Tokenize the text
    words = text.split()

    # 4. Remove stopwords
    words = [word for word in words if word not in stop_words]

    # 5. Lemmatize each word
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # 6. Join back into a string
    return ' '.join(lemmatized_words)

df = dataset0.copy()

df = df.drop_duplicates()

df["text"] = df["text"].apply(preprocess_text)
print(df.head())

                                                text  label
0                                          get bings      1
1                      tv female friend group belong      1
2      new star war force awakens trailer give chill      1
3  vine new york celebrity big brother fucking pe...      1
4  couple stunning photo shoot baby learning inop...      1


In [7]:
# Train/Test split

train_data, test_data, train_labels, test_labels = train_test_split(
    df["text"], df["label"], test_size=0.2, random_state=42)

# Further split train_data into train/val

train_data, val_data, train_labels, val_labels = train_test_split(
    train_data, train_labels, test_size=0.2, random_state=42)


print(train_data.shape)
print(test_data.shape)

(20480,)
(6400,)


In [8]:
# Terms Frequency -  Inverse Document Frequency


vectorizer = TfidfVectorizer(max_features=5000)

X_train = vectorizer.fit_transform(train_data).toarray()
X_val = vectorizer.transform(val_data).toarray()
X_test = vectorizer.transform(test_data).toarray()

In [25]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# === Compile with default settings (no optimizer tuning)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# === Train model (no early stopping)
model.fit(X_train, train_labels, validation_data=(X_val, val_labels), epochs=7)

accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred, zero_division=0)
recall = recall_score(test_labels, y_pred, zero_division=0)
precision = precision_score(test_labels, y_pred, zero_division=0)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")

/opt/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/7
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8026 - loss: 0.3707 - val_accuracy: 0.9473 - val_loss: 0.1442
Epoch 2/7
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9787 - loss: 0.0593 - val_accuracy: 0.9438 - val_loss: 0.1573
Epoch 3/7
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9875 - loss: 0.0356 - val_accuracy: 0.9445 - val_loss: 0.1820
Epoch 4/7
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9923 - loss: 0.0225 - val_accuracy: 0.9406 - val_loss: 0.2097
Epoch 5/7
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9957 - loss: 0.0130 - val_accuracy: 0.9387 - val_loss: 0.2388
Epoch 6/7
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9962 - loss: 0.0097 - val_accuracy: 0.9408 - val_loss: 0.2819
Epoch 7/7
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9975 - loss: 0.0061 - val_accuracy: 0.9428 - val_loss: 0.2935
Accuracy: 0.9505
F1-score: 0.9508
Recall: 0.9358
Precision: 0.9662


In [26]:
model.save("saved_models/nn_instance1.h5")
print("Model saved to saved_models/nn_instance2.h5")

Model saved to saved_models/nn_instance2.h5


In [9]:
# build neural network model

# Define model with regularization, dropout, deeper layers
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))  # Dropout added
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Compile with custom learning rate
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train with validation and early stopping
model.fit(X_train, train_labels, validation_data=(X_val, val_labels), epochs=20, callbacks=[early_stop])


test_tfidf=  vectorizer.transform(test_data).toarray()

y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")

# Optional: Print predictions and true labels
print("Predicted labels:", y_pred.flatten())
print("True labels:", test_labels.values)

/opt/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7929 - loss: 1.0518 - val_accuracy: 0.9285 - val_loss: 0.5421
Epoch 2/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9254 - loss: 0.5346 - val_accuracy: 0.9252 - val_loss: 0.5051
Epoch 3/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9277 - loss: 0.5055 - val_accuracy: 0.9197 - val_loss: 0.4950
Epoch 4/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9262 - loss: 0.4882 - val_accuracy: 0.9297 - val_loss: 0.4820
Epoch 5/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9329 - loss: 0.4747 - val_accuracy: 0.9287 - val_loss: 0.4656
Epoch 6/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9325 - loss: 0.4681 - val_accuracy: 0.9191 - val_loss: 0.4957
Epoch 7/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9296 - loss: 0.4576 - val_accuracy: 0.9291 - val_loss: 0.4617
Epoch 8/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9369 - loss: 0.4395 - val_accuracy: 0.

In [10]:
model.save("saved_models/nn_instance2.h5")
print("Model saved to saved_models/nn_instance2.h5")

Model saved to saved_models/nn_instance2.h5


In [14]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weight_dict = dict(enumerate(class_weights))
print("Class weights:", class_weight_dict)

# === Build the model ===
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# === Compile the model using RMSprop ===
optimizer = RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# === Early stopping ===
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# === Train the model ===
history = model.fit(
    X_train, train_labels,
    validation_data=(X_val, val_labels),
    epochs=20,
    callbacks=[early_stop],
    class_weight=class_weight_dict
)

y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred, zero_division=0)
recall = recall_score(test_labels, y_pred, zero_division=0)
precision = precision_score(test_labels, y_pred, zero_division=0)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")


Class weights: {0: np.float64(0.9940782448305989), 1: np.float64(1.0059927301306613)}


/opt/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8070 - loss: 0.5858 - val_accuracy: 0.9381 - val_loss: 0.2348
Epoch 2/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9463 - loss: 0.2133 - val_accuracy: 0.9406 - val_loss: 0.2182
Epoch 3/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9462 - loss: 0.1950 - val_accuracy: 0.9371 - val_loss: 0.2088
Epoch 4/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9456 - loss: 0.1900 - val_accuracy: 0.9244 - val_loss: 0.2303
Epoch 5/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9459 - loss: 0.1800 - val_accuracy: 0.9412 - val_loss: 0.2031
Epoch 6/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9495 - loss: 0.1775 - val_accuracy: 0.9354 - val_loss: 0.1958
Epoch 7/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9465 - loss: 0.1771 - val_accuracy: 0.9309 - val_loss: 0.2180
Epoch 8/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9459 - loss: 0.1745 - val_accuracy: 0.

In [12]:
model.save("saved_models/nn_instance3.h5")
print("Model saved to saved_models/nn_instance3.h5")

Model saved to saved_models/nn_instance3.h5




# === Build the model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# === Compile with SGD optimizer
optimizer = SGD(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# === Early stopping
early_stop = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# === Train the model
history = model.fit(
    X_train, train_labels,
    validation_data=(X_val, val_labels),
    epochs=30,
    callbacks=[early_stop],
    class_weight=class_weight_dict
)

y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred, zero_division=0)
recall = recall_score(test_labels, y_pred, zero_division=0)
precision = precision_score(test_labels, y_pred, zero_division=0)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")


In [22]:
model.save("saved_models/nn_instance4.h5")
print("Model saved to saved_models/nn_instance4.h5")

Model saved to saved_models/nn_instance4.h5


In [20]:

# === Build the model
logreg_model = LogisticRegression(
    max_iter=1000,
    C=1.0,  # Lower values = stronger regularization
    solver='liblinear'  # Good for small datasets and binary classification
)

# === Fit the model
logreg_model.fit(X_train, train_labels)

# === Predict
y_pred = logreg_model.predict(X_test)

# === Evaluate
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred, zero_division=0)
recall = recall_score(test_labels, y_pred, zero_division=0)
precision = precision_score(test_labels, y_pred, zero_division=0)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")

Accuracy: 0.9505
F1-score: 0.9508
Recall: 0.9358
Precision: 0.9662


In [21]:
joblib.dump(logreg_model, "saved_models/logreg_model.pkl")
print("Model saved to saved_models/logreg_model.pkl'")

Model saved to saved_models/logreg_model.pkl'
